In [97]:
import glob
import numpy as np
import scipy as sp
import einops as eo
import matplotlib.pyplot as plt
import math
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation
import magpylib as magpy
import os


M0 = 1480 #mT
shape = [3*25.4/16, 25.4/8] #radius and height
x0 = np.array([0,0,10, 0,0,1])

In [98]:
def B_dipole(position, rotation, M0, shape):
    R = np.sqrt(np.sum(position**2, axis=1))
    B = (M0 * (shape[0]) ** 2 * shape[1] / (16)) * (
        (
            3
            * position
            / R[:, np.newaxis] ** 5
            * (eo.einsum(position, rotation, "sensor dim,  dim -> sensor"))[
                :, np.newaxis
            ]
        )
        - rotation[np.newaxis, :] / (R[:, np.newaxis] ** 3)
    )
    return B


def getField_dipole(x, positions, M0, shape):
    position = x[:3]
    axis = x[3:]
    return B_dipole(positions - position, axis, M0, shape)


def getField_dipole_fixed(x, positions, M0, shape):
    position = x[:3]
    axis = x[3:]
    return B_dipole(positions - position, axis, M0, shape)


def cost_dipole(x, B, positions, M0, shape):
    diff = getField_dipole(x, positions, M0, shape) - B
    return np.sum((diff) ** 2)


def getField_cylinder(x, positions, M0, shape):
    B=magpy.getB(
        sources="Cylinder",
        position=x[:3],
        orientation=Rotation.align_vectors(x[3:], np.array([0, 0, 1]))[0],
        observers=positions,
        dimension=shape,
        polarization=(0, 0, M0),
    )
    return B

def cost_cylinder(x, B, positions, M0, shape):
    diff = getField_cylinder(x, positions, M0, shape) - B
    return np.sum((diff) ** 2)

def minimize(x0, B, positions, M0, shape, *args):
    #print("Starting mimimization")
    b_args = (B, positions, M0, shape)
    cons = [{"type": "eq", "fun": lambda x: x[3] ** 2 + x[4] ** 2 + x[5] ** 2 - 1}]
    bounds = [(-100, 100), (-100, 100), (0, 100), (-1, 1), (-1, 1), (-1, 1)]
    res = sp.optimize.minimize(
        fun=cost_dipole, x0=x0, args=b_args, tol=1e-100, constraints=cons, bounds=bounds, *args
    ).x  
    #print(f"Finished mimimization with shape {b_args[3]} at {res}")
    return res

def circle_radius(x1, y1, x2, y2, x3, y3):
    # Calculate lengths of sides of the triangle formed by the three points
    a = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    b = math.sqrt((x3 - x2) ** 2 + (y3 - y2) ** 2)
    c = math.sqrt((x3 - x1) ** 2 + (y3 - y1) ** 2)

    # Calculate the semi-perimeter of the triangle
    s = (a + b + c) / 2

    # Calculate the area of the triangle using Heron's formula
    area = math.sqrt(s * (s - a) * (s - b) * (s - c))

    # Calculate the radius of the circle using the formula: radius = (abc) / (4 * area)
    radius = (a * b * c) / (4 * area)
    
    return radius

def comparison_plot(locact, locpred, axis, relative):

    if axis == 'x':
        ax = 0
    elif axis == 'y':
        ax = 1
    else:
        ax = 2

    index = list(range(len(locact[:,ax])))
    act = locact[:, ax]
    pred = locpred[:, ax]

    rel = ''

    if relative==True:
        act = [x - locact[0, ax] for x in locact[:, ax]]
        pred = [x - locpred[0, ax] for x in locpred[:, ax]]
        rel = ' (Relative)'

    # Plotting
    plt.figure(figsize=(8, 6))
    plt.scatter(index, pred, label='Predicted' + rel)
    plt.scatter(index, act, label='Actual' + rel)
    plt.xlabel('Index')
    plt.ylabel(axis + '-component')
    plt.title('Comparison of '+axis+ ' Location' + rel+ ': Predicted vs Actual')
    plt.legend()
    plt.grid(True)
    plt.show()

    return

def comp_plot3D(locact, locpred, xact, yact, axis, relative):

    if axis == 'x':
        ax = 0
    elif axis == 'y':
        ax = 1
    else:
        ax = 2

    index = list(range(len(locact[:,ax])))
    act = locact[:, ax]
    pred = locpred[:, ax]

    rel = ''

    if relative==True:
        act = [x - locact[0, ax] for x in locact[:, ax]]
        pred = [x - locpred[0, ax] for x in locpred[:, ax]]
        rel = ' (Relative)'

    fig = go.Figure(data=[
        go.Scatter3d(x=xact, y=yact, z=pred, name='Predicted' + rel),
        go.Scatter3d(x=xact, y=yact, z=act, name='Actual' + rel)])
    fig.show()

    return

def displacement_plot(actual, predicted):

    dispx = np.array(predicted[:,0]-predicted[:,0][0])
    dispy = np.array(predicted[:,1]-predicted[:,1][0])
    dispz = np.array(predicted[:,2]-predicted[:,2][0])
    disp = np.sqrt(dispx**2+dispy**2+dispz**2)
    actdispx = np.array(actual[:,0]-actual[:,0][0])
    actdispy = np.array(actual[:,1]-actual[:,1][0])
    actdispz = np.array(actual[:,2]-actual[:,2][0])
    actdisp = np.sqrt(actdispx**2+actdispy**2+actdispz**2)

    plt.figure(figsize=(8, 6))
    plt.scatter(list(range(len(disp))), disp, label='Predicted Displacement')
    plt.scatter(list(range(len(actdisp))), actdisp, label='Actual Displacement')
    plt.xlabel('Index')
    plt.ylabel('Displacement (mm)')
    plt.title('Magnet Displacement')
    plt.legend()
    plt.grid(True)
    plt.show()

    return

def importfitting(folder, indices):

    file_list = sorted(glob.glob(f'{folder}/*.npz'))
    len(file_list)
    data = np.load(file_list[0])
    pos = data['pos'][0]
    
    
    locpred = [np.array([0,0,0])]
    anglepred = [np.array([0,0,0])]
    all_mags = [np.array([0,0,0])]
    Bstdev = [np.array(pos)]
    for x in range(len(file_list)):
        data = np.load(file_list[x])
        mags = data['mags']
        all_mags = np.append(all_mags, mags[0], axis=0)
        pos = data['pos'][0][indices]
        B = np.mean(mags[0][indices], axis=0)
        mags_stdev = np.std(mags, axis=0)
        x_res = minimize(x0, B, pos, M0, shape)
        locpred = np.append(locpred, [x_res[0:3]], axis=0)
        anglepred = np.append(anglepred, [x_res[3:6]], axis=0)
        Bstdev = np.append(Bstdev, [mags_stdev], axis=0)
    locpred = np.delete(locpred, 0, axis=0)
    anglepred = np.delete(anglepred, 0, axis=0)
    Bstdev = np.delete(Bstdev, 0, axis=0)
    all_mags = np.delete(all_mags, 0, axis=0)

    return pos, mags, locpred, anglepred, Bstdev

def actualangle(xind, yind, xscal, xcon, yscal, ycon, isoff):

    angact = [np.array([0, 0, 0])]
    for y in range(yind):
        for x in range(xind):
            R = Rotation.from_euler('ZYX' ,np.array([0, yscal*y-ycon, xscal*x-xcon]), degrees=True)
            norm = np.array([[0, 0, 1]])
            rotated = R.apply(norm, inverse=False)[0]
            #add = np.array([np.sin(np.deg2rad(2*y-4)), -np.sin(np.deg2rad(2*x-4)), np.cos(np.deg2rad(2*y-4))*np.cos(np.deg2rad(2*x-4))])
            angact = np.append(angact, [rotated], axis=0)
    angact = np.delete(angact, 0, axis=0)
    locact = angact*isoff

    return angact, locact

In [99]:
folder = os.path.join('..', './MLX06242024_D32N52_AboutZ_Logs')
x= range(17)
print(x)
pos, all_mags, locpred, anglepred, Bstdev = importfitting(folder, range(16))
np.append(locpred, anglepred, axis=1)

range(0, 17)


array([[-4.36295935e-01, -2.54540625e+01,  4.23011306e+00,
        -5.17137405e-03, -9.77431058e-01,  2.11191346e-01],
       [-4.37888814e-01, -2.54503757e+01,  4.22903174e+00,
        -5.18952512e-03, -9.77436332e-01,  2.11166489e-01],
       [-4.30827090e-01, -2.54571089e+01,  4.21519989e+00,
        -5.10625425e-03, -9.77592321e-01,  2.10445197e-01],
       [-4.37421822e-01, -2.54530906e+01,  4.22546177e+00,
        -5.18472982e-03, -9.77478049e-01,  2.10973418e-01],
       [-4.35567756e-01, -2.54489999e+01,  4.23906359e+00,
        -5.16196553e-03, -9.77329266e-01,  2.11662137e-01],
       [-4.34018738e-01, -2.54546691e+01,  4.22202032e+00,
        -5.14487222e-03, -9.77517010e-01,  2.10793800e-01],
       [-4.34732650e-01, -2.54544790e+01,  4.22618377e+00,
        -5.15297484e-03, -9.77473007e-01,  2.10997552e-01],
       [-4.37206975e-01, -2.54531423e+01,  4.23639482e+00,
        -5.18204725e-03, -9.77363524e-01,  2.11503399e-01],
       [-4.35975809e-01, -2.54509098e+01,  4.228

In [100]:
file_list = sorted(glob.glob(f'{folder}/*.npz'))
len(file_list)
data = np.load(file_list[0])
pos = data['pos'][0]

all_mags = [np.array([0,0,0])]
for x in range(len(file_list)):
    data = np.load(file_list[x])
    mags = data['mags']
    all_mags = np.append(all_mags, mags[0], axis=0)
    pos = data['pos'][0]
    B = np.mean(mags, axis=0)
    mags_stdev = np.std(mags, axis=0)
all_mags = np.delete(all_mags, 0, axis=0)

print(all_mags)
bx = all_mags[:,0]
by = all_mags[:,1]
bz = all_mags[:,2]

print(bx)
len(bx)


[[-1.5273   -1.7043    0.38236 ]
 [-1.1877   -3.1053    1.361492]
 [ 0.5067   -3.8502    1.777732]
 ...
 [-0.0633   -0.4137   -0.1573  ]
 [ 0.0681   -0.4626   -0.166496]
 [ 0.1158   -0.3927   -0.15246 ]]
[-1.5273 -1.1877  0.5067 ... -0.0633  0.0681  0.1158]


2000

In [101]:
bx7 = []
by7 = []
for x in range(2000):
    if x % 16 == 8:
        bx7.append(bx[x])
        by7.append(by[x])
bx7

[-0.258,
 -0.25859999999999994,
 -0.2562,
 -0.258,
 -0.25859999999999994,
 -0.258,
 -0.2562,
 -0.25739999999999996,
 -0.25709999999999994,
 -0.25830000000000003,
 -0.25830000000000003,
 -0.25709999999999994,
 -0.25739999999999996,
 -0.2595,
 -0.25739999999999996,
 -0.25859999999999994,
 -0.2367,
 -0.1893,
 -0.0006,
 -0.0057,
 0.0033,
 0.4125,
 1.1076,
 2.7662999999999998,
 3.5403,
 3.8505,
 2.3891999999999998,
 1.2843,
 1.2951,
 0.7965,
 -0.32939999999999997,
 -0.5829,
 -0.5816999999999999,
 -0.5709,
 -0.462,
 -0.4311,
 -0.4269,
 -0.4206,
 -0.4023,
 -0.3693,
 -0.3558,
 -0.3408,
 -0.32280000000000003,
 -0.32430000000000003,
 -0.3168,
 -0.3039,
 -0.2934,
 -0.2703,
 -0.2001,
 -0.1515,
 -0.09989999999999999,
 -0.10289999999999999,
 -0.1062,
 -0.1038,
 0.2049,
 1.0743,
 1.0943999999999998,
 1.716,
 2.3291999999999997,
 3.846,
 1.3818,
 0.1092,
 -0.5237999999999999,
 -0.5373,
 -0.5408999999999999,
 -0.47759999999999997,
 -0.4305,
 -0.3624,
 -0.3444,
 -0.33030000000000004,
 -0.3195,
 -0.30419

In [102]:
index = np.array(range(2000))
print(index)
measuredbx = go.Scatter(x=index, y=bx7, mode='markers')
layoutbx = go.Layout(title='b_x over time')
figbx = go.Figure(data=[measuredbx], layout=layoutbx)
figbx.update_yaxes(scaleanchor='y')
figbx.update_layout(
    xaxis_title="Index",
    yaxis_title="b_x [mT]"
)
figbx.show()

[   0    1    2 ... 1997 1998 1999]


In [103]:
measuredby = go.Scatter(x=index, y=by7, mode='markers')
layoutby = go.Layout(title='b_y over time')
figby = go.Figure(data=[measuredby], layout=layoutby)
figby.update_yaxes(scaleanchor='y')
figby.update_layout(
    xaxis_title="Index",
    yaxis_title="b_y [mT]"
)
figby.show()

In [104]:
print(anglepred)
angles = np.arcsin(anglepred)*180/np.pi
print(angles)
#print(locpred)

[[-5.17137405e-03 -9.77431058e-01  2.11191346e-01]
 [-5.18952512e-03 -9.77436332e-01  2.11166489e-01]
 [-5.10625425e-03 -9.77592321e-01  2.10445197e-01]
 [-5.18472982e-03 -9.77478049e-01  2.10973418e-01]
 [-5.16196553e-03 -9.77329266e-01  2.11662137e-01]
 [-5.14487222e-03 -9.77517010e-01  2.10793800e-01]
 [-5.15297484e-03 -9.77473007e-01  2.10997552e-01]
 [-5.18204725e-03 -9.77363524e-01  2.11503399e-01]
 [-5.16695400e-03 -9.77439393e-01  2.11152872e-01]
 [-5.14543850e-03 -9.77458662e-01  2.11064182e-01]
 [-5.17692474e-03 -9.77477549e-01  2.10975924e-01]
 [-5.19491212e-03 -9.77434235e-01  2.11176061e-01]
 [-5.11555453e-03 -9.77517811e-01  2.10790796e-01]
 [-5.16265172e-03 -9.77485286e-01  2.10940423e-01]
 [-4.42470294e-03 -9.77409721e-01  2.11307026e-01]
 [-3.51302674e-03 -9.76554063e-01  2.15243631e-01]
 [ 7.83561959e-03 -9.77720437e-01  2.09764989e-01]
 [ 2.71596618e-02 -9.74497187e-01  2.22750050e-01]
 [ 1.29179727e-01 -9.32730638e-01  3.36639502e-01]
 [ 1.34852323e-01 -9.29785856e-

In [105]:
Dist = 13.77 #mm
SensorHeight = 0.85
zact = Dist - shape[1]/2 - SensorHeight
radact = np.sqrt(800)/2
point = (radact, 0, zact)

# Function to rotate a 3D point around the z-axis
def rotate_point_z_axis(point, angle_degrees):
    angle_radians = np.radians(angle_degrees)
    rotation_matrix = np.array([
        [np.cos(angle_radians), -np.sin(angle_radians), 0],
        [np.sin(angle_radians), np.cos(angle_radians), 0],
        [0, 0, 1]
    ])
    return np.dot(rotation_matrix, point)

# Generate points rotating every 60 degrees
angles = [60 * i for i in range(6)]
locact = np.array([rotate_point_z_axis(point, angle) for angle in angles])

# Print the points
for i, p in enumerate(locact):
    print(f"Point {i}: {p}")
    
actang = np.array([(0, 0, -1) for _ in range(len(locact))])

Point 0: [14.14213562  0.         11.3325    ]
Point 1: [ 7.07106781 12.24744871 11.3325    ]
Point 2: [-7.07106781 12.24744871 11.3325    ]
Point 3: [-1.41421356e+01  1.73191211e-15  1.13325000e+01]
Point 4: [ -7.07106781 -12.24744871  11.3325    ]
Point 5: [  7.07106781 -12.24744871  11.3325    ]


In [106]:
size = 10000

measured = go.Cone(x=locpred[:,0], y=locpred[:,1], z=locpred[:,2], u=anglepred[:,0], v=anglepred[:,1], w=anglepred[:,2], 
                             sizemode="absolute", sizeref=size, colorscale=[(0, 'red'), (1, 'red')])

calculated = go.Cone(x=locact[:,0], y=locact[:,1], z=locact[:,2], u=actang[:,0], v=actang[:,1], w=actang[:,2], 
                             sizemode="absolute", sizeref=0.01, colorscale=[(0, 'blue'), (1, 'blue')])

layout = go.Layout(title='Location Prediction', scene=dict(aspectmode='data', 
    xaxis_title='x [mm]', yaxis_title='y (mm)', zaxis_title='z (mm)'))
fig = go.Figure(data=[measured], layout=layout)

fig.show()

In [107]:
a = 0
b = 1

measured2D = go.Scatter(x=locpred[:,a], y=locpred[:,b], mode='markers')
calc2D = go.Scatter(x=locact[:,a], y=locact[:,b], mode='markers')
layout = go.Layout(title='x vs y (measured)')
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_layout(
    xaxis_title="x [mm]",
    yaxis_title="y [mm]"
)
fig2.update_yaxes(scaleanchor='x')
fig2.show()

In [108]:
a = 0
b = 1

measured2D = go.Scatter3d(x=locpred[:,a], y=locpred[:,b], z=locpred[:,2], mode='markers', marker=dict(
        size=12,
        color=np.array(range(len(locpred[:,a]))),                # set color to an array/list of desired values
        colorbar=dict(
            title="Index"
        ),
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    ))
#calc2D = go.Scatter(x=locact[:,a], y=locact[:,b], mode='markers')
layout = go.Layout(title='Rotation over Time', scene=dict(xaxis_title='x [mm]', yaxis_title='y [mm]', zaxis_title='z [mm]'))
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_yaxes(scaleanchor='x')
fig2.show()

In [109]:
a = 0
b = 2

measured2D = go.Scatter(x=locpred[:,a], y=locpred[:,b], mode='markers')
layout = go.Layout(title='x vs z (measured)', scene=dict(xaxis_title='x [mm]', yaxis_title='z [mm]'))
calc2D = go.Scatter(x=locact[:,a], y=locact[:,b], mode='markers')
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_yaxes(scaleanchor='x')
fig2.update_layout(
    xaxis_title="x [mm]",
    yaxis_title="z [mm]"
)
fig2.show()

In [110]:
a = 1
b = 2

measured2D = go.Scatter(x=locpred[:,a], y=locpred[:,b], mode='markers')
layout = go.Layout(title='y vs z (measured)', scene=dict(xaxis_title='y [mm]', yaxis_title='z [mm]'))
calc2D = go.Scatter(x=locact[:,a], y=locact[:,b], mode='markers')
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_yaxes(scaleanchor='x')
fig2.update_layout(
    xaxis_title="y [mm]",
    yaxis_title="z [mm]"
)
fig2.show()